In [ ]:
# import os
# from dotenv import load_dotenv
# from crewai import Agent, Task, Crew, Process
# from crewai_tools import SerperDevTool

In [1]:
import os
from dotenv import load_dotenv
from crewai import Agent, Task, Process, Crew
from langchain.utilities import GoogleSerperAPIWrapper
from langchain.agents import Tool
from langchain.agents import load_tools

In [2]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
SERPER_API_KEY = os.getenv("SERPER_API_KEY")

In [ ]:
# GAME: 
# NY Yankees - Carlos Rodon
# TOR Blue Jays - Yusei Kikuchi

# RUN LINE: 
# NY Yankees: -1.5 (+142)
# TOR Blue Jays: +1.5 (-170)

# TOTAL:
# O 8.5 (-118)
# U 8.5 (-102)

# MONEY LINE:
# NY Yankees: -115
# TOR Blue Jays: -105


In [3]:
search = GoogleSerperAPIWrapper()

search_tool = Tool(
    name="Scrape Google Searches",
    func=search.run,
    description="Useful for when you need to ask the agent to search the internet"
)


# Or
# search_tool = SerperDevTool()

In [ ]:
# My Agents

os.environ["OPENAI_MODEL_NAME"]="gpt-4-turbo-preview"


# Agent 1: 
# 'Research teams deeply from upcoming games for later use in modeling and predictions'

# backstory = '''Research the upcoming game between {team1} and {team2}, including general and advanced
# team statistics for both of them as well as who is starting pitching for them both'''

In [4]:
# Full GPT Example

team1 = input("team 1: ")
team2 = input("team 2: ")

# Define agents with backstories
research_agent = Agent(
    role="Baseball Data Researcher",
    goal="Gather historical and current season data on baseball teams",
    tools=[search_tool],
    verbose=True,
    backstory="A seasoned sports analyst with a passion for baseball statistics and game theory."
)

prediction_agent = Agent(
    role="Prediction Analyst",
    goal="Analyze baseball game data to predict game outcomes",
    tools=[search_tool],
    verbose=True,
    backstory="A data-driven forecaster with a strong record in predicting sports outcomes based on statistical models."
)

output_agent = Agent(
    role="Report Generator",
    goal="Generate a readable prediction report",
    tools=[search_tool],
    verbose=True,
    backstory="An expert in communicating complex data insights through clear and engaging reports."
)

# Define tasks
research_task = Task(
    description=f"Collect the latest stats, player conditions, and historical performance for {team1} and {team2}.",
    expected_output=f"A dataset with all relevant baseball game data for {team1} and {team2}",
    agent=research_agent,
    output_file='./logs/research_output.txt'
)

prediction_task = Task(
    description=f"Analyze the data collected to predict the outcome of the next baseball game between {team1} and {team2}.",
    expected_output=f"""A prediction of the winner and score of the baseball game between {team1} and {team2}, including a
    final score and the top performers with stats of each team (home runs, hits, RBI's)""",
    agent=prediction_agent,
    context=[research_task],
    output_file='./logs/prediction_output.txt'
)

player_stats_task = Task(
    description=f"Predict individual batting stats for players of {team1} and {team2} based on current season data and player conditions.",
    expected_output=f"Predictions of home runs, hits, RBI's, and slugging percentages for the top 4 key players for both {team1} and {team2}.",
    agent=prediction_agent,
    context=[research_task, prediction_task],  # Depends on data from both research and prediction tasks
    output_file='./logs/player_stats_output.txt'
)

# player_stats_report = """
# **Batting Stats Predictions for Upcoming Game**

# Los Angeles Dodgers key player predictions:
# - Freddie Freeman: Batting Average (BA) .305, On-Base Percentage (OBP) .380, Slugging Percentage (SLG) .525
# - Teoscar Hernández: BA .280, OBP .340, SLG .500
# - Shohei Ohtani: BA .290, OBP .360, SLG .540

# Washington Nationals key player predictions:
# - Juan Soto: BA .315, OBP .430, SLG .550
# - Trea Turner: BA .295, OBP .350, SLG .480
# - Josh Bell: BA .270, OBP .340, SLG .470
# """



output_task = Task(
    description="Format the prediction into a comprehensive report.",
    expected_output="A well-formatted report summarizing the prediction.",
    agent=output_agent,
    context=[prediction_task],
    output_file='./logs/final_report.txt'
)

# Create the crew
baseball_crew = Crew(
    agents=[research_agent, prediction_agent, output_agent],
    tasks=[research_task, prediction_task, player_stats_task, output_task],
    process=Process.sequential,
    verbose=True
)

# Execute tasks
result = baseball_crew.kickoff()
print(result)

team 1: Dodgers
team 2: Nationals
 [DEBUG]: == Working Agent: Baseball Data Researcher
 [INFO]: == Starting Task: Collect the latest stats, player conditions, and historical performance for Dodgers and Nationals.


> Entering new CrewAgentExecutor chain...
Thought: To gather the latest stats, player conditions, and historical performance for the Dodgers and Nationals, I need to start by collecting the most recent and comprehensive data available. I should utilize web scraping to gather current season statistics and player conditions. For historical performance, I will need data that spans several years, possibly decades, to analyze trends and performance metrics effectively.

Action: Scrape Google Searches
Action Input: {"query": "Los Angeles Dodgers 2023 season statistics"} 

The Los Angeles Dodgers were 100-62 in 2023. They finished 1st in the NL West.

Thought: With the 2023 season statistics for the Dodgers now known, I need to do the same for the Nationals to ensure I have the lat

Thought: 
The search results provided a very brief overview of the Washington Nationals' performance in their last 10 games, indicating a 4-6 record and a team batting average of .275, but did not provide the comprehensive statistics I need. There's no detailed data like individual player statistics, home runs, hits, RBI's, or information on recent injuries or changes in team composition for either the Dodgers or the Nationals. To make an accurate prediction for the game between the Dodgers and Nationals, including a final score and top performers, I would need much more specific data.

Final Answer: 
I was unable to gather the detailed recent performance data for both the Los Angeles Dodgers and the Washington Nationals, including their last 10 games, head-to-head matchups this season, and individual player statistics such as batting averages, home runs, hits, RBI's. Additionally, I could not obtain information on recent injuries or changes in team composition that would be crucial fo

Thought: Now that I have detailed statistics for the top performers on the Dodgers, I need similar information for the Nationals to compare both teams' strengths and make a prediction. This data is crucial for understanding each team's offensive capabilities and will significantly influence the prediction of the game's outcome, including the final score and top performers.

Action: Ask question to co-worker
Action Input: {"coworker": "Baseball Data Researcher", "question": "Can you provide the latest available statistics for the top three performers on the Nationals?", "context": "To complete my analysis and predict the outcome of the next baseball game between the Dodgers and Nationals, I require detailed statistical information about the top performers on the Nationals, similar to what was provided for the Dodgers. This includes their recent batting averages, home runs, hits, and RBI's. Focusing on the top three performers will offer insight into the team's offensive capabilities, wh

Thought: Given the detailed statistics for the Dodgers' top performers and the home run leaders for the Nationals, I have a good starting point for predicting the game outcome. However, the lack of detailed batting averages, hits, and RBIs for the Nationals' top performers is a significant gap. Given the data at hand, I can still formulate a prediction based on the Dodgers' strong offensive capabilities as indicated by their top performers and the notable home run hitters for the Nationals. The Dodgers seem to have a more robust lineup in terms of batting averages and home runs, which could give them an edge in the upcoming game. 

Final Answer: Based on the available statistics and performance data, the prediction for the next baseball game between the Dodgers and Nationals leans towards a victory for the Dodgers. The Dodgers' top performers, particularly Freddie Freeman and Mookie Betts, who have shown impressive stats in batting averages, home runs, and RBIs, suggest a strong offens

Action Input: {"query": "Lane Thomas 2023 season statistics"} 

Led the final roster of Nationals with a career-high 17 home runs and a .404 slugging percentage while setting career marks in nearly every offensive category... ... 2024 season stats · AVG .213. 143rd · HR. 2. Tied-74th · RBI. 10. Tied-42nd · OPS .591. 150+. Lane Thomas. Position: Outfielder. Bats: Right • Throws: Right. 6-0, 195lb (183cm, 88kg). Team: Washington Nationals (majors). On the road, Lane Thomas appeared in 80 games, had 330 at-bats, drove in 35 runs, scored 50 times, had 75 hits (17 doubles / 2 triples / 13 home runs), walked ... 2024 season stats ; AVG .176. 150+ ; HR. 0. 150+ ; RBI. 4. Tied-104th ; OPS .451. 150+. Lane Thomas had a .312 batting average with 93 hits, 15 home runs, 51 RBIs and 51 runs scored in 76 games at home in 2023. Lane Thomas pitched in 1 Major League game during the 2023 regular season. The chart below is a comprehensive analysis of the games Thomas pitched in, ... Lane Thomas 2023 bat

Thought: To find the latest season statistics for a fourth key player from the Nationals who is a batter, I need to identify who that player is first, since the task specifies a player other than Freddie Freeman, Mookie Betts, and Lane Thomas. After identifying the player, I can proceed to gather the required statistics on home runs, hits, RBI's, and slugging percentages.

Action: Scrape Google Searches
Action Input: {"query": "Washington Nationals 2023 key batters"} 

MacKenzie Gore, one of the key pieces from the Soto trade, has a chance to break out. Josiah Gray, one of the pieces from the Max Scherzer ... Exciting new features at Nationals Park for the 2023 baseball season, including grab-and-go beer stands, new food vendors, theme nights, ... The key for García will be not getting complacent in that knowledge. Stone Garrett — A-. Garrett's breakout season ended in a painful way, ... Missing: battery | Show results with:battery. Show your team spirit with an officially licensed MLB

In [ ]:
# Trend Finder Agent

researcher = Agent(
  role='Senior Research Analyst',
  goal='Uncover cutting-edge developments in AI and data science',
  backstory="""You work at a leading tech think tank.
  Your expertise lies in identifying emerging trends.
  You have a knack for dissecting complex data and presenting actionable insights.""",
  verbose=True,

In [ ]:
# Pitcher+Batter Matchup Analysis Agent

backstory = '''
Find and research all the historical matchups between a batter and
pitcher to see if there are any trends in the matchup.
'''

In [ ]:
# Team Basic Agents
# Replace 'sportsdata' with actual data source implementations

task1 = Task(
    description="Analyze team's past 5 games performance",
    agent=team_performance_agent,
    data_source="sportsdata.get_team_performance(team_id, last_n_games=5)"
)

task2 = Task(
    description="Evaluate key player injuries",
    agent=player_injury_agent,
    data_source="sportsdata.get_player_injuries(team_id)"
)

task3 = Task(
    description="Review past matchups between the two teams",
    agent=match_history_agent,
    data_source="sportsdata.get_match_history(team1_id, team2_id)"
)
